In [18]:
import pandas as pd
from backend.data_center.kline_data.kline_data_collector import *
tv = KlineDataCollector()
file_abspath = tv.get_abspath(symbol='BTC', interval=Interval.in_daily)
df = pd.read_csv(f"{file_abspath}")
df

,datetime,symbol,open,high,low,close,volume
0,2017-08-17 08:00:00,Binance:BTCUSDT,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 08:00:00,Binance:BTCUSDT,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 08:00:00,Binance:BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 08:00:00,Binance:BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 08:00:00,Binance:BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...,...
2604,2024-10-03 08:00:00,Binance:BTCUSDT,60649.27,61477.19,59828.11,60752.71,26224.819320
2605,2024-10-04 08:00:00,Binance:BTCUSDT,60752.72,62484.85,60459.90,62086.00,21299.146910
2606,2024-10-05 08:00:00,Binance:BTCUSDT,62086.00,62370.56,61689.26,62058.00,7848.591930
2607,2024-10-06 08:00:00,Binance:BTCUSDT,62058.01,62975.00,61798.97,62819.91,8961.253090


In [29]:
# df['is_20d_high'] = df['high'] == df['high'].expanding(min_periods=3).max()

df['is_20d_high'] = (df['high'] == df['high'].rolling(window=20, min_periods=1).max())

In [30]:
df['20d_high'] = df['high'].rolling(window=20, min_periods=1).max()
df[df['is_20d_high']==True]
# [df['is_20d_high']==True]

,datetime,symbol,open,high,low,close,volume,is_20d_high,20d_high
0,2017-08-17 08:00:00,Binance:BTCUSDT,4261.48,4485.39,4200.74,4285.08,795.150377,True,4485.39
12,2017-08-29 08:00:00,Binance:BTCUSDT,4353.65,4625.85,4313.55,4587.48,603.545028,True,4625.85
13,2017-08-30 08:00:00,Binance:BTCUSDT,4564.52,4647.51,4416.01,4555.14,808.468771,True,4647.51
14,2017-08-31 08:00:00,Binance:BTCUSDT,4555.14,4745.42,4555.14,4724.89,556.956802,True,4745.42
15,2017-09-01 08:00:00,Binance:BTCUSDT,4689.89,4885.55,4654.88,4834.91,560.666366,True,4885.55
...,...,...,...,...,...,...,...,...,...
2591,2024-09-20 08:00:00,Binance:BTCUSDT,62948.00,64133.32,62350.00,63201.05,25466.377940,True,64133.32
2594,2024-09-23 08:00:00,Binance:BTCUSDT,63578.76,64745.88,62538.75,63339.99,24078.052870,True,64745.88
2596,2024-09-25 08:00:00,Binance:BTCUSDT,64262.70,64817.99,62947.08,63152.01,17813.111680,True,64817.99
2597,2024-09-26 08:00:00,Binance:BTCUSDT,63152.01,65839.00,62670.00,65173.99,28373.305930,True,65839.00
